In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from skimage import color

In [2]:
data_transform = transforms.Compose([
    transforms.RandomResizedCrop(64, scale=(0.08, 1.)),
    transforms.RandomHorizontalFlip()
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
])
data_folder='../data'
train = datasets.STL10(root=data_folder, transform=data_transform)

# trainset = datasets.MNIST(root='.', train=True, download=True, transform=data_transform)
# testset = datasets.MNIST(root='.', train=False, download=True, transform=data_transform)


In [3]:
# train_dataset = datasets.ImageFolder(data_folder+'/unsupervised_test', transform=data_transform)
train_dataset = datasets.ImageFolder(data_folder+'/train', transform=data_transform)

In [4]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 5000
    Root location: ../data/train

In [89]:
vars(train_dataset)
# train_dataset

{'root': '../data/unsupervised_test', 'transform': Compose(
     RandomResizedCrop(size=(64, 64), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
     RandomHorizontalFlip(p=0.5)
 ), 'target_transform': None, 'transforms': None, 'loader': <function torchvision.datasets.folder.default_loader(path)>, 'extensions': ('.jpg',
  '.jpeg',
  '.png',
  '.ppm',
  '.bmp',
  '.pgm',
  '.tif',
  '.tiff',
  '.webp'), 'classes': ['0'], 'class_to_idx': {'0': 0}, 'samples': [('../data/unsupervised_test/0/0.png',
   0),
  ('../data/unsupervised_test/0/1.png', 0),
  ('../data/unsupervised_test/0/10.png', 0),
  ('../data/unsupervised_test/0/100.png', 0),
  ('../data/unsupervised_test/0/1000.png', 0),
  ('../data/unsupervised_test/0/1001.png', 0),
  ('../data/unsupervised_test/0/1002.png', 0),
  ('../data/unsupervised_test/0/1003.png', 0),
  ('../data/unsupervised_test/0/1004.png', 0),
  ('../data/unsupervised_test/0/1005.png', 0),
  ('../data/unsupervised_test/0/1006.png', 0),
 

In [5]:
img = []
minL = np.inf 
maxL = -np.inf
minA = np.inf 
maxA = -np.inf
minB = np.inf 
maxB = -np.inf
for i in train_dataset:
#     plt.imshow(i)
    img = i
#     a,b = i
#     print(np.mean(color.rgb2lab(np.asarray(img[0], np.uint8)), axis=(0,1)))
#     print(np.max(color.rgb2lab(np.asarray(img[0], np.uint8)), axis=(0,1)))
#     print(np.min(color.rgb2lab(np.asarray(img[0], np.uint8)), axis=(0,1)))
    maxLab = np.max(color.rgb2lab(np.asarray(img[0], np.uint8)), axis=(0,1))
    maxL = max(maxL, maxLab[0])
    maxA = max(maxA, maxLab[1])    
    maxB = max(maxB, maxLab[2])
    
    minLab = np.min(color.rgb2lab(np.asarray(img[0], np.uint8)), axis=(0,1))
    minL = min(minL, minLab[0])
    minA = min(minA, minLab[1])    
    minB = min(minB, minLab[2])
    
#     print(a.shape)
#     print(a.permute((1,2,0)).shape)
#     img = a.permute((1,2,0))
#     plt.imshow(a.permute((1,2,0)))
#     break

In [6]:
maxL, maxA, maxB, minL, minA, minB

(100.0,
 85.69985867746288,
 94.47812227647825,
 0.0,
 -70.73682919640534,
 -89.82557819975)

In [ ]:
# 90,000 unlabeled dataset
(100.0,
 90.78142512637616,
 94.47812227647825,
 0.0,
 -84.91423369614904,
 -107.85730020669489)

In [52]:
img[0], np.asarray(img[0], np.uint8).shape

(<PIL.Image.Image image mode=RGB size=64x64 at 0x7FBE3020FF60>, (64, 64, 3))

In [57]:
np.mean(color.rgb2lab(np.asarray(img[0], np.uint8)), axis=(0,1))

array([45.54325952,  0.22511904, 20.72737707])

In [ ]:
train_set.data.mean(axis=(0,1,2))/255

In [50]:
plt.imshow(color.rgb2lab(img))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Image'

In [33]:
img.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [35]:
img.dtype

torch.float32

In [12]:
img[0][0]

tensor([0.5725, 0.5608, 0.4314])

In [21]:
np.asarray(img, np.uint8)[0][0]

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [14]:
color.rgb2lab(np.asarray(img, np.uint8))[0][0]

array([0., 0., 0.])

In [19]:
color.rgb2lab(img)[0][0]

array([58.86510042, -4.5252827 , 18.01362536])

In [50]:
np.asarray(img, np.uint8).dtype

dtype('uint8')

In [43]:
color.rgb2lab(img).dtype

dtype('float64')

In [51]:
color.rgb2lab(np.asarray(img, np.uint8)).dtype

dtype('float64')

In [59]:
np.float32(color.rgb2lab(img)).dtype

dtype('float32')

In [ ]:
class RGB2Lab(object):
    """Convert RGB PIL image to ndarray Lab."""
    def __call__(self, img):

        img = np.asarray(img, np.uint8)
        img = color.rgb2lab(img)
        return img



In [100]:
from __future__ import print_function

import torch
import torch.nn as nn

"""
    Alexnet for STL-10
"""
class alexnet(nn.Module):
    def __init__(self, feat_dim=64):
        super(alexnet, self).__init__()

        self.l_to_ab = alexnet_half(in_channel=1, feat_dim=feat_dim)
        self.ab_to_l = alexnet_half(in_channel=2, feat_dim=feat_dim)

    def forward(self, x, layer=8):
        l, ab = torch.split(x, [1, 2], dim=1)
        feat_l = self.l_to_ab(l, layer)
        feat_ab = self.ab_to_l(ab, layer)
        return feat_l, feat_ab


class alexnet_half(nn.Module):
    def __init__(self, in_channel=1, feat_dim=128):
        super(alexnet_half, self).__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channel, 96//2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(96//2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2),
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(96//2, 192//2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(192//2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2),
        )
        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(192//2, 384//2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(384//2),
            nn.ReLU(inplace=True),
        )
        self.conv_block_4 = nn.Sequential(
            nn.Conv2d(384//2, 384//2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(384//2),
            nn.ReLU(inplace=True),
        )
        self.conv_block_5 = nn.Sequential(
            nn.Conv2d(384//2, 192//2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(192//2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2),
        )
        self.fc6 = nn.Sequential(
            nn.Linear(192 * 7 * 7 // 2, 4096 // 2),
            nn.BatchNorm1d(4096 // 2),
            nn.ReLU(inplace=True),
        )
        self.fc7 = nn.Sequential(
            nn.Linear(4096 // 2, 4096 // 2),
            nn.BatchNorm1d(4096 // 2),
            nn.ReLU(inplace=True),
        )
        self.fc8 = nn.Sequential(
            nn.Linear(4096 // 2, feat_dim)
        )
        self.l2norm = Normalize(2)

    def forward(self, x, layer):
        if layer <= 0:
            return x
        x = self.conv_block_1(x)
        if layer == 1:
            return x
        x = self.conv_block_2(x)
        if layer == 2:
            return x
        x = self.conv_block_3(x)
        if layer == 3:
            return x
        x = self.conv_block_4(x)
        if layer == 4:
            return x
        x = self.conv_block_5(x)
        if layer == 5:
            return x
        x = x.view(x.shape[0], -1)
        x = self.fc6(x)
        if layer == 6:
            return x
        x = self.fc7(x)
        if layer == 7:
            return x
        x = self.fc8(x)
        x = self.l2norm(x)
        return x


class Normalize(nn.Module):

    def __init__(self, power=2):
        super(Normalize, self).__init__()
        self.power = power

    def forward(self, x):
        norm = x.pow(self.power).sum(1, keepdim=True).pow(1. / self.power)
        out = x.div(norm)
        return out

In [101]:
from torchsummary import summary

In [102]:
model = alexnet()
summary(model, (3,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 48, 64, 64]             432
       BatchNorm2d-2           [-1, 48, 64, 64]              96
              ReLU-3           [-1, 48, 64, 64]               0
         MaxPool2d-4           [-1, 48, 31, 31]               0
            Conv2d-5           [-1, 96, 31, 31]          41,472
       BatchNorm2d-6           [-1, 96, 31, 31]             192
              ReLU-7           [-1, 96, 31, 31]               0
         MaxPool2d-8           [-1, 96, 15, 15]               0
            Conv2d-9          [-1, 192, 15, 15]         165,888
      BatchNorm2d-10          [-1, 192, 15, 15]             384
             ReLU-11          [-1, 192, 15, 15]               0
           Conv2d-12          [-1, 192, 15, 15]         331,776
      BatchNorm2d-13          [-1, 192, 15, 15]             384
             ReLU-14          [-1, 192,

In [ ]:
model.

In [ ]:
if __name__ == '__main__':

    import torch
    # model = alexnet().cuda()
    model = alexnet()
    # data = torch.rand(10, 3, 224, 224).cuda()
    data = torch.rand(10, 3, 224, 224)
    out = model.compute_feat(data, 5)

    for i in range(10):
        out = model.compute_feat(data, i)
        print(i, out.shape)

In [80]:

class alexnet(nn.Module):
    def __init__(self, feat_dim=128):
        super(alexnet, self).__init__()

        self.l_to_ab = alexnet_half(in_channel=1, feat_dim=feat_dim)
        self.ab_to_l = alexnet_half(in_channel=2, feat_dim=feat_dim)

    def forward(self, x, layer=8):
        l, ab = torch.split(x, [1, 2], dim=1)
        feat_l = self.l_to_ab(l, layer)
        feat_ab = self.ab_to_l(ab, layer)
        return feat_l, feat_ab


class alexnet_half(nn.Module):
    def __init__(self, in_channel=1, feat_dim=128):
        super(alexnet_half, self).__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channel, 96//2, 11, 4, 2, bias=False),
            nn.BatchNorm2d(96//2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2),
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(96//2, 256//2, 5, 1, 2, bias=False),
            nn.BatchNorm2d(256//2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2),
        )
        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(256//2, 384//2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(384//2),
            nn.ReLU(inplace=True),
        )
        self.conv_block_4 = nn.Sequential(
            nn.Conv2d(384//2, 384//2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(384//2),
            nn.ReLU(inplace=True),
        )
        self.conv_block_5 = nn.Sequential(
            nn.Conv2d(384//2, 256//2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(256//2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2),
        )
        self.fc6 = nn.Sequential(
            nn.Linear(256 * 6 * 6 // 2, 4096 // 2),
            nn.BatchNorm1d(4096 // 2),
            nn.ReLU(inplace=True),
        )
        self.fc7 = nn.Sequential(
            nn.Linear(4096 // 2, 4096 // 2),
            nn.BatchNorm1d(4096 // 2),
            nn.ReLU(inplace=True),
        )
        self.fc8 = nn.Sequential(
            nn.Linear(4096 // 2, feat_dim)
        )
        self.l2norm = Normalize(2)

    def forward(self, x, layer):
        if layer <= 0:
            return x
        x = self.conv_block_1(x)
        if layer == 1:
            return x
        x = self.conv_block_2(x)
        if layer == 2:
            return x
        x = self.conv_block_3(x)
        if layer == 3:
            return x
        x = self.conv_block_4(x)
        if layer == 4:
            return x
        x = self.conv_block_5(x)
        if layer == 5:
            return x
        x = x.view(x.shape[0], -1)
        x = self.fc6(x)
        if layer == 6:
            return x
        x = self.fc7(x)
        if layer == 7:
            return x
        x = self.fc8(x)
        x = self.l2norm(x)
        return x


class Normalize(nn.Module):

    def __init__(self, power=2):
        super(Normalize, self).__init__()
        self.power = power

    def forward(self, x):
        norm = x.pow(self.power).sum(1, keepdim=True).pow(1. / self.power)
        out = x.div(norm)
        return out

In [81]:
model = alexnet()

In [85]:
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 48, 55, 55]           5,808
       BatchNorm2d-2           [-1, 48, 55, 55]              96
              ReLU-3           [-1, 48, 55, 55]               0
         MaxPool2d-4           [-1, 48, 27, 27]               0
            Conv2d-5          [-1, 128, 27, 27]         153,600
       BatchNorm2d-6          [-1, 128, 27, 27]             256
              ReLU-7          [-1, 128, 27, 27]               0
         MaxPool2d-8          [-1, 128, 13, 13]               0
            Conv2d-9          [-1, 192, 13, 13]         221,184
      BatchNorm2d-10          [-1, 192, 13, 13]             384
             ReLU-11          [-1, 192, 13, 13]               0
           Conv2d-12          [-1, 192, 13, 13]         331,776
      BatchNorm2d-13          [-1, 192, 13, 13]             384
             ReLU-14          [-1, 192,

In [3]:
ckpt = torch.load('out/model/memory_nce_4096_alexnet_lr_0.03_decay_0.0001_bsz_256/ckpt_epoch_10.pth')

In [11]:
ckpt['model']['l_to_ab.conv_block_1.0.weight'].shape

torch.Size([48, 1, 3, 3])

In [12]:
for i in ckpt['model']:
    print(i)

l_to_ab.conv_block_1.0.weight
l_to_ab.conv_block_1.1.weight
l_to_ab.conv_block_1.1.bias
l_to_ab.conv_block_1.1.running_mean
l_to_ab.conv_block_1.1.running_var
l_to_ab.conv_block_1.1.num_batches_tracked
l_to_ab.conv_block_2.0.weight
l_to_ab.conv_block_2.1.weight
l_to_ab.conv_block_2.1.bias
l_to_ab.conv_block_2.1.running_mean
l_to_ab.conv_block_2.1.running_var
l_to_ab.conv_block_2.1.num_batches_tracked
l_to_ab.conv_block_3.0.weight
l_to_ab.conv_block_3.1.weight
l_to_ab.conv_block_3.1.bias
l_to_ab.conv_block_3.1.running_mean
l_to_ab.conv_block_3.1.running_var
l_to_ab.conv_block_3.1.num_batches_tracked
l_to_ab.conv_block_4.0.weight
l_to_ab.conv_block_4.1.weight
l_to_ab.conv_block_4.1.bias
l_to_ab.conv_block_4.1.running_mean
l_to_ab.conv_block_4.1.running_var
l_to_ab.conv_block_4.1.num_batches_tracked
l_to_ab.conv_block_5.0.weight
l_to_ab.conv_block_5.1.weight
l_to_ab.conv_block_5.1.bias
l_to_ab.conv_block_5.1.running_mean
l_to_ab.conv_block_5.1.running_var
l_to_ab.conv_block_5.1.num_batche

In [15]:
ckpt['model']['ab_to_l.fc8.0.weight'].shape

torch.Size([128, 2048])

In [41]:
from __future__ import print_function

import torch.nn as nn


class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, feat):
        return feat.view(feat.size(0), -1)


class LinearClassifierAlexNet(nn.Module):
    def __init__(self, layer=5, n_label=1000, pool_type='max'):
        super(LinearClassifierAlexNet, self).__init__()
        if layer == 1:
            pool_size = 10
            nChannels = 96
        elif layer == 2:
            pool_size = 6
            nChannels = 256
        elif layer == 3:
            pool_size = 5
            nChannels = 384
        elif layer == 4:
            pool_size = 5
            nChannels = 384
        elif layer == 5:
            pool_size = 7
            nChannels = 192
        else:
            raise NotImplementedError('layer not supported: {}'.format(layer))

        self.classifier = nn.Sequential()

        if layer < 5:
            if pool_type == 'max':
                self.classifier.add_module('MaxPool', nn.AdaptiveMaxPool2d((pool_size, pool_size)))
            elif pool_type == 'avg':
                self.classifier.add_module('AvgPool', nn.AdaptiveAvgPool2d((pool_size, pool_size)))

        self.classifier.add_module('Flatten', Flatten())
        self.classifier.add_module('LinearClassifier', nn.Linear(nChannels*pool_size*pool_size, n_label))
        self.initilize()

    def initilize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.fill_(0.0)

    def forward(self, x):
        return self.classifier(x)


class LinearClassifierResNetV2(nn.Module):
    def __init__(self, layer=6, n_label=1000, pool_type='avg'):
        super(LinearClassifierResNetV2, self).__init__()
        if layer == 1:
            pool_size = 8
            nChannels = 128 * 2
            pool = pool_type
        elif layer == 2:
            pool_size = 6
            nChannels = 256 * 2
            pool = pool_type
        elif layer == 3:
            pool_size = 4
            nChannels = 512 * 2
            pool = pool_type
        elif layer == 4:
            pool_size = 3
            nChannels = 1024 * 2
            pool = pool_type
        elif layer == 5:
            pool_size = 7
            nChannels = 2048 * 2
            pool = pool_type
        elif layer == 6:
            pool_size = 1
            nChannels = 2048 * 2
            pool = pool_type
        else:
            raise NotImplementedError('layer not supported: {}'.format(layer))

        self.classifier = nn.Sequential()
        if layer < 5:
            if pool == 'max':
                self.classifier.add_module('MaxPool', nn.AdaptiveMaxPool2d((pool_size, pool_size)))
            elif pool == 'avg':
                self.classifier.add_module('AvgPool', nn.AdaptiveAvgPool2d((pool_size, pool_size)))
        else:
            # self.classifier.add_module('AvgPool', nn.AvgPool2d(7, stride=1))
            pass

        self.classifier.add_module('Flatten', Flatten())
        print('classifier input: {}'.format(nChannels * pool_size * pool_size))
        self.classifier.add_module('LiniearClassifier', nn.Linear(nChannels * pool_size * pool_size, n_label))
        self.initilize()

    def initilize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.fill_(0.0)

    def forward(self, x):
        return self.classifier(x)


In [44]:
from torchsummary import summary

model = LinearClassifierAlexNet()


summary(model, (2,64,64))

RuntimeError: size mismatch, m1: [2 x 8192], m2: [9408 x 1000] at /Users/distiller/project/conda/conda-bld/pytorch_1556653464916/work/aten/src/TH/generic/THTensorMath.cpp:961

# Split Validation Set

In [9]:
data_folder='../data/'

In [10]:
t0 = np.load(data_folder+'train_images_0.npy')

In [11]:
t0

array(['data/train/39.png', 'data/train/4948.png', 'data/train/2964.png',
       'data/train/2855.png', 'data/train/1866.png',
       'data/train/1306.png', 'data/train/2954.png',
       'data/train/1040.png', 'data/train/2450.png',
       'data/train/1829.png', 'data/train/533.png', 'data/train/2241.png',
       'data/train/4218.png', 'data/train/1070.png',
       'data/train/3840.png', 'data/train/2508.png',
       'data/train/4582.png', 'data/train/2461.png', 'data/train/947.png',
       'data/train/4510.png', 'data/train/4769.png', 'data/train/548.png',
       'data/train/734.png', 'data/train/183.png', 'data/train/3036.png',
       'data/train/1472.png', 'data/train/1720.png',
       'data/train/3102.png', 'data/train/3905.png',
       'data/train/4535.png', 'data/train/101.png', 'data/train/4371.png',
       'data/train/1604.png', 'data/train/3714.png',
       'data/train/2551.png', 'data/train/4585.png', 'data/train/131.png',
       'data/train/3648.png', 'data/train/3180.png',
